#**Ingesta a través de API rest**

#**Configuración de AWS CLI en Colab**


In [6]:
##Instalación de boto3 & request para la API de los datos
!pip install boto3
!pip install requests

In [7]:
import requests
import boto3
import json
from io import StringIO
import pandas as pd

In [8]:
##Parametros de conexión con AWS###
!pip install awscli
!pip install --upgrade awscli

aws_access_key_id='____________________'
aws_secret_access_key='________________________________________'
aws_session_token='_____________________________________________________________'

!aws configure list --profile assumed-role

      Name                    Value             Type    Location
      ----                    -----             ----    --------
   profile             assumed-role           manual    --profile

The config profile (assumed-role) could not be found


In [9]:
##Parametros del bucket##
s3_bucket='rawzonetrabajo1'
s3_target_file_name='raw/atm_con.csv'

#**Obteniendo la información del API que tiene el IMF**

####**Annual Surface Temperature Change**

In [11]:
url_atm_con = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_2_Climate_Indicators_Monthly_Atmospheric_Carbon_Dioxide_concentrations/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

response_atm_con = requests.get(url_atm_con)
data_atm_con= response_atm_con.json()

features_atm_con = data_atm_con['features']

df_atm_con= pd.json_normalize(features_atm_con, record_path=None, meta=['id', 'geometry'], errors='ignore')

df_atm_con = pd.DataFrame([feature.get('properties', {}) for feature in features_atm_con])